#***Question*** 


DDPG is frequently brittle to HyperParameters, why?
1.   Firstly due to the high dimentional Paramter space associated with the model, such that it is hard to find the optimal combination of these. 

***Tricks that we use in TD DDPG*** 



1.   We use double Q-learning 
2.   We update the Policy and the target-functions less frequently than the Q-function 
3.   Add noise to the target Action





#***Why will Double Q-Learning Help?*** 

Recall exactly what Q-learning tries to do... 
We want to find: 

***maxₐE[Q(s,a)];***

Meaning we want to find the action that on average maximises the expected reward in the future. 

Usually we try to estimate it using normal Q-learning, which is something like: 

***E[maxₐQ(s,a)]***

Which can be shown to have the following relationship with what we are trying to estimate 

***E[maxₐQ(s,a)] >= maxₐE[Q(s,a)];***

But now that we are using Double-Q-Learning we have the following TD-Update: 

***Qₐ'(s,t) <- Qₐ(s, a) + (r+Qᵦ(s', *argmax*Qₐ(s', a')) - Qₐ(s, a)***

Comparing to our original gaol we get: 

***E[Qₐ(s,E[argmaxₐQᵦ(s,a)])] <= maxₐE[Q(s,a)]***

I am not sure of the proof, but we can walk through this logically: 

1.   Let us say we have 2 Q-value functions, now there will be a discounted sum of rewards distribution for every action.
2.   On the right hand side we calculate the maximum expected sum of rewards that any action can give us.
3.   On the left we take some action which will have the largest sum of rewards distribution ***(according to what we have seen)***
4.   We take the expected value of this action. 
5.   We know the right hand side already has the maximum expected value of the discounted reward for all actions in the given state.
6. Thus if we take the Expected value of any Q-function the expected discounted reward will be less or equal to this expected value even if Qᵦ states that some action a' will have the highest return. 


We can now see that when we use only a single q-value function, we will get over-estimation, which will lead us to change the q-value more than we would have wanted to in the direction of some action. 

So we will not use this double q-learning in the same way here as we did for the TD updated, but we will be able to say, let us calculate some Q-value for 2 different Q networks(Constructed the same, but trained on different data) and choose the smaller of the 2. This is called clipped double-q-learning. 


#***Why will Delayed Policy Updates Help?***

The target networks are used to provide stability in training. 
When we are using some Q-network to find values, it might happen that we over-estimat our policy. This happens because in DDPG we couple the output of the critic to be the input of the value function, such that we can estimate the value of that action. 
using single Q-networks, we might overestimate our action and so we might update our policy by the wrong amount and more imortantly in the wrong direction. Thus, if we estimate that a bad policy will do well, in this case, our learning might diverge. 

Our agent's policy will continue to get worse as it updates on states with large error. Thus, we allow the value network to become more stable before we update the critic network. 


#***Why will adding noise to the target action help?***

We want to smooth out the Q-values over similar actions, such that we don't have the divergent behaviour we spoke of, above. If we add noisee to the action, this means that for similar states we might get different actions, meaning that the policy is not as deterministic anymore. In the DDPG case we did not add noise to the target network, this meant that eventhough we are mittigating the spike from happening in the actor network it can still occur in the target actor network. 